In [1]:
import pickle
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [2]:
with open("preprocessed_data.pickle", "rb") as fr:
     preprocessed_data = pickle.load(fr)

In [3]:
test_inputs = preprocessed_data['test_x']
test_outputs = preprocessed_data['test_y']
token = preprocessed_data['token']
w_l = preprocessed_data['sentence_len']
st = preprocessed_data['stop_words']
tk = preprocessed_data['token']

In [4]:
loaded_model = load_model('best_m.h5')
loaded_model.evaluate(test_inputs, test_outputs)

929/929 [==============================] - 20s 20ms/step - loss: 0.4723 - accuracy: 0.7774


[0.4722881019115448, 0.7773849368095398]

In [5]:
def print_result(t):
    otk = Okt()
    t.replace('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]', '')
    t = otk.morphs(t)
    t = [w for w in t if w not in st]
    t = tk.texts_to_sequences([t])
    t = pad_sequences(t, maxlen = w_l)
    score = float(loaded_model.predict(t))
    
    if score > 0.5:
        print(f"{score * 100:.2f}% 확률로 긍정 리뷰입니다.\n")
    else:
        print(f"{(1 - score) * 100:.2f}% 확률로 부정 리뷰입니다.\n")

In [6]:
print_result('반지의 제왕 좋아해서 샀고 지인과 함께 플레이 했습니다. 싱글로는 글쎄요... 스토리는 재밌었지만 그래픽도 그닥이고 볼륨도 그닥이고 지루하기 짝이 없습니다.')

95.50% 확률로 부정 리뷰입니다.



In [7]:
loaded_model.layers

In [8]:
len(loaded_model.layers[0].get_weights())

1

In [9]:
len(loaded_model.layers[0].get_weights()[0])

12559

In [10]:
len(loaded_model.layers[0].get_weights()[0][1])

64

In [11]:
d = token.word_index.items()

In [12]:
for w, i in d:
    print(i)
    break

4


In [13]:
pr_data = sorted(d, key = lambda x : x[1])

In [14]:
for w, i in pr_data:
    print(i)
    break

1


In [15]:
len(token.word_index)

59246

In [16]:
w_l_data = list(token.word_index.keys())

In [17]:
w_l_data.insert(0,'OOV')

In [18]:
w_l_data = w_l_data[:12559]

In [19]:
ws = loaded_model.layers[0].get_weights()[0]

In [20]:
len(ws), len(w_l_data)

(12559, 12559)

In [21]:
import io
out_v = io.open('v_data.tsv', 'w', encoding = 'utf-8')
out_w = io.open('w_data.tsv', 'w', encoding = 'utf-8')
for i, w in enumerate(w_l_data):
    if i == 0:
        continue
    vec = ws[i]
    out_v.write('\t'.join([str(i) for i in vec]) + '\n')
    out_w.write(w + '\n')
out_v.close()
out_w.close()